In [ ]:
! pip install deap
! pip install scoop

     |████████████████████████████████| 160 kB 4.8 MB/s 
     |████████████████████████████████| 603 kB 5.3 MB/s 
  Created wheel for scoop: filename=scoop-0.7.1.1-py3-none-any.whl size=72141 sha256=9dd2862f37fd170a28ea130aedfbac5d663820ec7e0c8afa95a7738cc9280120
  Stored in directory: /root/.cache/pip/wheels/24/19/e9/6e3e7c0323cc36bf1e4993d69b2db27d6b4e806ed57d411f44
Successfully built scoop


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn.model_selection as model_selection
import pickle, scipy, csv, statistics, math, warnings, joblib, random, numpy
from sklearn import metrics, svm, datasets
from math import log,e, floor
from time import time
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score,cross_val_predict,GridSearchCV, train_test_split, StratifiedKFold, KFold, cross_validate, learning_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import plot_confusion_matrix, mean_absolute_error,accuracy_score,r2_score,confusion_matrix ,classification_report, f1_score, precision_score, recall_score
from sklearn import metrics, preprocessing, svm
from sklearn.linear_model import LogisticRegression
from deap import creator, base, tools, algorithms
from scoop import futures
from sklearn.utils import shuffle
warnings.filterwarnings("ignore")
import sys
import warnings
if not sys.warnoptions:
  warnings.simplefilter("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Sequential methods for channel selection/Subject Independent/data files/Wavelet Based/

/content/drive/MyDrive/Sequential methods for channel selection/Subject Independent/data files/Wavelet Based


In [ ]:
!ls

s01_arousal.csv      s11_new_arousal.csv  s23_arousal.csv
s01_FourClass.csv    s11_valence.csv	  s23_FourClass.csv
s01_new_arousal.csv  s12_arousal.csv	  s23_new_arousal.csv
s01_valence.csv      s12_FourClass.csv	  s23_valence.csv
s02_arousal.csv      s12_new_arousal.csv  s24_arousal.csv
s02_FourClass.csv    s12_valence.csv	  s24_FourClass.csv
s02_new_arousal.csv  s13_arousal.csv	  s24_new_arousal.csv
s02_valence.csv      s13_FourClass.csv	  s24_valence.csv
s03_arousal.csv      s13_new_arousal.csv  s25_arousal.csv
s03_FourClass.csv    s13_valence.csv	  s25_FourClass.csv
s03_new_arousal.csv  s14_arousal.csv	  s25_new_arousal.csv
s03_valence.csv      s14_FourClass.csv	  s25_valence.csv
s04_arousal.csv      s14_new_arousal.csv  s26_arousal.csv
s04_FourClass.csv    s14_valence.csv	  s26_FourClass.csv
s04_new_arousal.csv  s15_arousal.csv	  s26_new_arousal.csv
s04_valence.csv      s15_FourClass.csv	  s26_valence.csv
s05_arousal.csv      s15_new_arousal.csv  s27_arousal.csv
s05_FourClass.csv 

In [ ]:
def emotion_label(labels, class_label):
	em_labels = []
	if(class_label == "valence"):
		for i in range(0, labels.shape[0]):
			if (labels[i][0]>5): # high valence
				em_labels.append(1)
			else: # low valence
				em_labels.append(0)
		return em_labels
	elif(class_label == "arousal"):
		for i in range(0, labels.shape[0]):
			if (labels[i][1]>5): # high arousal
				em_labels.append(1)
			else: # low arousal
				em_labels.append(0)
		return em_labels
	elif(class_label == "all"):
		for i in range(0, labels.shape[0]):
			if (labels[i][0]>5): # high valence
				if(labels[i][1]>5): # high arousal
					em_labels.append(1) # HVHA
				else:
					em_labels.append(0) # HVLA
			else: # low valence
				if(labels[i][1]>5): # high arousal
					em_labels.append(2) # LVHA
				else: # low arousal
					em_labels.append(3) # LVLA
		return em_labels

In [ ]:
def kfold(allFeatures, allClasses, index):
    m = int(allClasses.shape[0])
    s = int(m/10)
    train_index = np.ones(m-s+1).astype(int)
    test_index  = np.ones(s).astype(int)
    for i in range(s*index,s*(index+1)):
        test_index[i-s*index] = i
    for i in range(0,s*index):
        train_index[i] = i
    for i in range(s + s*index,m):
        train_index[i-s] = i
    X_train = allFeatures.iloc[train_index]
    X_test = allFeatures.iloc[test_index]
    y_train = allClasses[train_index]
    y_test = allClasses[test_index]
    # normalize the xtrain data only with class labels
    return X_train, X_test, y_train, y_test

In [ ]:
# Feature subset fitness function
def getFitness(individual, X_train, X_test, y_train, y_test):
    """
    parse our feature columns that we don't use
    apply one hot encoding to the features.
    """
    cols = [index for index in range(len(individual)) if individual[index] == 0]
    X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
    X_trainOhFeatures = pd.get_dummies(X_trainParsed)
    X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
    X_testOhFeatures = pd.get_dummies(X_testParsed)

    # Remove any columns that aren't in both the training and test sets
    sharedFeatures = set(X_trainOhFeatures.columns) & set(X_testOhFeatures.columns)
    removeFromTrain = set(X_trainOhFeatures.columns) - sharedFeatures
    removeFromTest = set(X_testOhFeatures.columns) - sharedFeatures
    X_trainOhFeatures = X_trainOhFeatures.drop(list(removeFromTrain), axis=1)
    X_testOhFeatures = X_testOhFeatures.drop(list(removeFromTest), axis=1)

    # Apply logistic regression on the data, and calculate accuracy
    clf = svm.SVC(kernel='poly').fit(X_trainOhFeatures, y_train)
    predictions = clf.predict(X_testOhFeatures)
    accuracy = accuracy_score(y_test, predictions)*100
    return (accuracy,)

In [ ]:
def getFitness_all(individual, X_train, X_test, y_train, y_test):
    """
    Feature subset fitness function
    Parse our feature columns that we don't use
    Apply one hot encoding to the features.
    """
    cols = [index for index in range(len(individual)) if individual[index] == 0]
    #print(cols)
    #print(cols.shape)
    X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
    X_trainOhFeatures = pd.get_dummies(X_trainParsed)
    X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
    X_testOhFeatures = pd.get_dummies(X_testParsed)

    # Remove any columns that aren't in both the training and test sets
    sharedFeatures = set(X_trainOhFeatures.columns) & set(X_testOhFeatures.columns)
    removeFromTrain = set(X_trainOhFeatures.columns) - sharedFeatures
    removeFromTest = set(X_testOhFeatures.columns) - sharedFeatures
    X_trainOhFeatures = X_trainOhFeatures.drop(list(removeFromTrain), axis=1)
    X_testOhFeatures = X_testOhFeatures.drop(list(removeFromTest), axis=1)

    # Apply logistic regression on the data, and calculate accuracy
    clf = svm.SVC(kernel='poly').fit(X_trainOhFeatures, y_train)
    predictions = clf.predict(X_testOhFeatures)
    accuracy = accuracy_score(y_test, predictions)*100
    return (accuracy, )

def getFinal(individual, X_train, X_test, y_train, y_test):
    """
    Feature subset fitness function
    Parse our feature columns that we don't use
    Apply one hot encoding to the features.
    """
    cols = [index for index in range(len(individual)) if individual[index] == 0]
    #print(cols)
    #print(cols.shape)
    X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
    X_trainOhFeatures = pd.get_dummies(X_trainParsed)
    X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
    X_testOhFeatures = pd.get_dummies(X_testParsed)

    # Remove any columns that aren't in both the training and test sets
    sharedFeatures = set(X_trainOhFeatures.columns) & set(X_testOhFeatures.columns)
    removeFromTrain = set(X_trainOhFeatures.columns) - sharedFeatures
    removeFromTest = set(X_testOhFeatures.columns) - sharedFeatures
    X_trainOhFeatures = X_trainOhFeatures.drop(list(removeFromTrain), axis=1)
    X_testOhFeatures = X_testOhFeatures.drop(list(removeFromTest), axis=1)

    # Apply logistic regression on the data, and calculate accuracy
    clf = svm.SVC(kernel='poly').fit(X_trainOhFeatures, y_train)
    predictions = clf.predict(X_testOhFeatures)
    accuracy = accuracy_score(y_test, predictions)*100
    ps = precision_score(y_test, predictions)*100
    rs = recall_score(y_test, predictions)*100
    f1 = f1_score(y_test, predictions)*100
    return accuracy, ps, rs, f1

In [ ]:
def getHof(toolbox):
    # Initialize variables to use eaSimple
    numPop = 100
    numGen = 50
    pop = toolbox.population(n=numPop)
    hof = tools.HallOfFame(numPop * numGen)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    # Launch genetic algorithm
    # change the crossover and mutation probability
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.65, mutpb=0.001, ngen=numGen, stats=stats, halloffame=hof, verbose=False)
    # Return the hall of fame
    return hof,log

In [ ]:
def getMetrics(hof, X_train, X_test, y_train, y_test):
    # Get list of percentiles in the hall of fame
    percentileList = [i / (len(hof) - 1) for i in range(len(hof))]
    # Gather fitness data from each percentile
    testAccuracyList = []
    validationAccuracyList = []
    individualList = []
    for individual in hof:
        testAccuracy = individual.fitness.values
        validationAccuracy = getFitness(individual, X_train, X_test, y_train, y_test)
        testAccuracyList.append(testAccuracy[0])
        validationAccuracyList.append(validationAccuracy[0])
        individualList.append(individual)
    #testAccuracyList.reverse()
    #validationAccuracyList.reverse()
    return testAccuracyList, validationAccuracyList, individualList, percentileList

In [ ]:
def drive(subject_name):
  dfData = pd.read_csv(subject_name + '_new_arousal.csv')
  allFeatures = dfData
  names = allFeatures.columns
  scaler = MinMaxScaler()
  allFeatures = scaler.fit_transform(allFeatures)
  allFeatures = pd.DataFrame(allFeatures, columns=names)
  allFeatures = allFeatures.loc[:, allFeatures.apply(pd.Series.nunique) != 1]
  link = "/content/drive/MyDrive/Deap/" + subject_name + ".dat"
  with open(link, 'rb') as f:
    raw_data = pickle.load(f, encoding = 'latin1')
  labels = raw_data['labels']
  em_labels = emotion_label(labels, 'arousal') # get the emotion labels
  allClasses = np.array(em_labels)
  allFeatures, allClasses = shuffle(allFeatures, allClasses, random_state = 40)
  return allFeatures, allClasses

In [ ]:
subject_names = ["s01", "s02", "s03", "s04", "s05", "s06", "s07", "s08", "s09", "s10", "s11", "s12",
                 "s13", "s14", "s15", "s16", "s17", "s18", "s19", "s20", "s21",
                "s22", "s23", "s24", "s25", "s26", "s27", "s28", "s29", "s30", "s31", "s32"]

In [ ]:
def main_code(allFeatures, allClasses):
  print("\nFold\tAccuracy_with_all_features\tAccuracy\tPrecision\tRecall\tF1-Score")
  acc_fs, pre_fs, recall_fs, f1_fs, all_acc = np.ones(10), np.ones(10), np.ones(10), np.ones(10), np.ones(10)
  for i in range(0, 10):
    #call k fold
    X_train, X_test, y_train, y_test = kfold(allFeatures, allClasses, i)
    #==========================    DEAP GLOBAL VARIABLES (viewable by SCOOP)      ======================
    # Create Individual
    creator.create("FitnessMax", base.Fitness, weights = (1.0,))
    creator.create("Individual", list, fitness = creator.FitnessMax)
    # Create Toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(allFeatures.columns) - 1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    # Continue filling toolbox...
    toolbox.register("evaluate", getFitness, X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb = 0.05)
    toolbox.register("select", tools.selTournament, tournsize = 7)
    #===================================================================================================
    # First, we will apply SVM using all the features to acquire a baseline accuracy.
    individual = [1 for i in range(len(allFeatures.columns))]
    Accuracy = getFitness_all(individual, X_train, X_test, y_train, y_test)
    all_acc[i] = Accuracy[0]
    if(all_acc[i]==100):
      acc_fs[i], pre_fs[i], recall_fs[i], f1_fs[i]  = Accuracy[0], Accuracy[0], Accuracy[0], Accuracy[0]
      print(i+1,"\t\t%.2f"%all_acc[i], "\t\t\t%.2f"%acc_fs[i], "\t\t%.2f"%pre_fs[i], "\t\t%.2f"%recall_fs[i], "\t%.2f"%f1_fs[i])
      continue
    #=====================================================================================================
    # Now, we will apply a genetic algorithm to choose a subset of features that gives a better accuracy than the baseline.
    hof, log = getHof(toolbox)
    best_indivisual = list(hof)[0]
    accuracy, ps, rs, f1 = getFinal(best_indivisual, X_train, X_test, y_train, y_test)
    if(ps == 0):
      ps, rs, f1 = accuracy, accuracy, accuracy
    acc_fs[i], pre_fs[i], recall_fs[i], f1_fs[i] = accuracy, ps, rs, f1
    print(i+1,"\t\t%.2f"%all_acc[i], "\t\t\t%.2f"%acc_fs[i], "\t\t%.2f"%pre_fs[i], "\t\t%.2f"%recall_fs[i], "\t%.2f"%f1_fs[i])
  print("-"*100)
  print("Total","\t\t%.2f"%np.mean(all_acc), "\t\t\t%.2f"%np.mean(acc_fs), "\t\t%.2f"%np.mean(pre_fs), "\t\t%.2f"%np.mean(recall_fs), "\t%.2f"%np.mean(f1_fs))
  print("-"*100)
  print("\n")

In [ ]:
for i in range(20,32):
  print('-'*100)
  print("Subject No: ", subject_names[i])
  print('-'*100)
  allFeatures, allClasses = drive(subject_names[i])
  main_code(allFeatures, allClasses)

----------------------------------------------------------------------------------------------------
Subject No:  s21
----------------------------------------------------------------------------------------------------

Fold	Accuracy_with_all_features	Accuracy	Precision	Recall	F1-Score
1 		75.00 			75.00 		75.00 		100.00 	85.71
2 		50.00 			75.00 		66.67 		100.00 	80.00
3 		75.00 			75.00 		75.00 		100.00 	85.71
4 		50.00 			50.00 		66.67 		66.67 	66.67
5 		100.00 			100.00 		100.00 		100.00 	100.00
6 		75.00 			75.00 		75.00 		100.00 	85.71
7 		75.00 			100.00 		100.00 		100.00 	100.00
8 		75.00 			75.00 		75.00 		100.00 	85.71
9 		100.00 			100.00 		100.00 		100.00 	100.00
10 		100.00 			100.00 		100.00 		100.00 	100.00
----------------------------------------------------------------------------------------------------
Total 		77.50 			82.50 		83.33 		96.67 	88.95
----------------------------------------------------------------------------------------------------


------------------

In [ ]:
for i in range(8,12):
  print('-'*100)
  print("Subject No: ", subject_names[i])
  print('-'*100)
  allFeatures, allClasses = drive(subject_names[i])
  main_code(allFeatures, allClasses)

In [ ]:
for i in range(24, 32):
  print('-'*100)
  print("Subject No: ", subject_names[i])
  print('-'*100)
  allFeatures, allClasses = drive(subject_names[i])
  main_code(allFeatures, allClasses)

**Accuracy**

In [ ]:
import numpy as np
accuracy = [87.50, 75.00, 82.50, 72.50, 70.00, 80.00, 87.50, 75.00, 70.00, 82.50, 72.50, 90.00, 90.00, 82.50, 80.00, 87.50, 77.50, 80.00, 72.50, 82.50,\
            80.00, 72.50, 92.50, 90.00, 72.50, 70.00, 72.50, 87.50, 80.00, 85.00, 85.00, 82.50]
print(len(accuracy))
accuracy = np.array(accuracy)
print(np.mean(accuracy))
print(np.std(accuracy))

32
80.234375
6.741875173820337


**Precision**

In [ ]:
precision = [83.33, 80.00, 85.00, 78.333, 76.67, 80.00, 91.67, 85.83, 80.83, 85.83, 70.00, 89.17, 90.00, 85.83, 86.67, 87.50, 86.67, 83.33, 73.33,\
             84.17, 81.67, 78.33, 95.00, 89.17, 78.33, 71.67, 77.50, 88.33, 85.00, 91.67, 89.17, 85.83]
print(len(precision))
precision = np.array(precision)
print(np.mean(precision))
print(np.std(precision))

32
83.61978125
5.899005656328738


**Recall or sensitivity**

In [ ]:
recall = [95.00, 87.50, 80.00, 71.67, 74.17, 86.67, 91.67, 83.33, 72.50, 86.67, 77.5, 100.00, 100.00, 93.33, 82.50, 90.00, 83.33, 94.17, 85.00, 96.67,\
          96.67, 86.67, 90.00, 100.00, 87.50, 80.00, 90.00, 93.33, 89.17, 81.67, 85.83, 94.17]
recall = np.array(recall)
print(np.mean(recall))
np.std(recall)

87.70906250000002


7.620516452058466

In [ ]:
[95.00, 87.50, 80.00, 71.67, 74.17, 86.67, 91.67, 83.33, 72.50, 86.67, 77.5, 100.00, 100.00, 93.33, 82.50, 90.00, 83.33, 94.17, 85.00, 96.67, 96.67, 86.67, 90.00, 100.00, 87.50, 80.00, 90.00, 93.33, 89.17, 81.67, 85.83, 94.17]

**f1-Score**

In [ ]:
f1 = [87.67, 78.90, 81.77, 72.00, 71.67, 81.00, 90.48, 80.4, 71.81, 84.24, 70.24, 93.71, 93.81, 88.38, 80.24, 85.24, 81.33, 85.90, 76.79, 88.38, 87.62,\
      76.57, 91.67, 92.00, 80.05, 71.67, 80.05, 89.33, 84.38, 84.33, 86.17, 88.48]
f1 = np.array(f1)
print(np.mean(f1))
print(np.std(f1))

83.00874999999999
6.722352522554884
